In [2]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

Encoder Model:

In [4]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

Decoder Model:

In [5]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

Now let's train our autoencoder to reconstruct MNIST digits.

First, we'll configure our model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer:

In [6]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Let's prepare our input data. We're using MNIST digits, and we're discarding the labels (since we're only interested in encoding/decoding the input images).

In [7]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

11493376/11490434 [==============================] - 5s 0us/step


We will normalize all values between 0 and 1 and we will flatten the 28x28 images into vectors of size 784.

In [9]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


Now let's train our autoencoder for 50 epochs:

In [10]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 5s 81us/step - loss: 0.3850 - val_loss: 0.2735
Epoch 2/50
60000/60000 [==============================] - 4s 72us/step - loss: 0.2663 - val_loss: 0.2558
Epoch 3/50
60000/60000 [==============================] - 4s 71us/step - loss: 0.2447 - val_loss: 0.2314
Epoch 4/50
60000/60000 [==============================] - 4s 70us/step - loss: 0.2229 - val_loss: 0.2127
Epoch 5/50
60000/60000 [==============================] - 4s 71us/step - loss: 0.2075 - val_loss: 0.2001
Epoch 6/50
60000/60000 [==============================] - 4s 71us/step - loss: 0.1967 - val_loss: 0.1907
Epoch 7/50
60000/60000 [==============================] - 4s 72us/step - loss: 0.1883 - val_loss: 0.1831
Epoch 8/50
60000/60000 [==============================] - 4s 72us/step - loss: 0.1814 - val_loss: 0.1768
Epoch 9/50
60000/60000 [==============================] - 4s 72us/step - loss: 0.1755 - val_loss: 0.1712
Epoch

After 50 epochs, the autoencoder seems to reach a stable train/test loss value of about 0.11. We can try to visualize the reconstructed inputs and the encoded representations. We will use Matplotlib.

In [11]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [15]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\lines.py:44: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if is_string_like(style) and is_hashable(style):
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\font_manager.py:971: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if is_string_like(family):
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\font_manager.py:697: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if is_string_like(family):
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\text.py:218: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  elif is_string_like(fontproperties):


AttributeError: module 'matplotlib.cbook' has no attribute 'popall'